In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
154,AFG,Asia,Afghanistan,2020-06-12,22890,747,426,21,588.003,19.189,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50
250,ALB,Europe,Albania,2020-06-12,1385,44,35,1,481.270,15.289,...,13.188,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89
410,DZA,Africa,Algeria,2020-06-12,10589,105,741,9,241.477,2.394,...,6.211,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-11,France,155573.0,0.0,29394.0,0.0,0.0,0.0
0,2020-06-11,China,84267.0,0.0,4651.0,0.0,0.0,0.0
0,2020-06-11,Italy,236178.0,0.0,34212.0,0.0,0.0,0.0
0,2020-06-11,Spain,242589.0,0.0,27147.0,0.0,0.0,0.0
0,2020-06-11,United States,2022037.0,0.0,114035.0,0.0,0.0,0.0
0,2020-06-11,World,7489721.0,0.0,421731.0,0.0,0.0,0.0
0,2020-06-11,United Kingdom,292218.0,0.0,41447.0,0.0,0.0,0.0
0,2020-06-11,Germany,185815.0,0.0,8784.0,0.0,0.0,0.0
0,2020-06-11,Iran,180347.0,0.0,8583.0,0.0,0.0,0.0
0,2020-06-11,Turkey,173986.0,0.0,4763.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-06-12,France,155573.0,155561.0,29394.0,29346.0,12.0,48.0
0,2020-06-12,China,84267.0,84216.0,4651.0,4638.0,51.0,13.0
0,2020-06-12,Italy,236178.0,236142.0,34212.0,34167.0,36.0,45.0
0,2020-06-11,Spain,242589.0,242707.0,27147.0,27136.0,-118.0,11.0
0,2020-06-12,United States,2022037.0,2023347.0,114035.0,113820.0,-1310.0,215.0
0,2020-06-12,World,7489721.0,7481063.0,421731.0,421190.0,8658.0,541.0
0,2020-06-12,United Kingdom,292218.0,291409.0,41447.0,41279.0,809.0,168.0
0,2020-06-12,Germany,185815.0,185674.0,8784.0,8763.0,141.0,21.0
0,2020-06-12,Iran,180347.0,180176.0,8583.0,8584.0,171.0,-1.0
0,2020-06-12,Turkey,173986.0,174023.0,4763.0,4763.0,-37.0,0.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")